# Probabilistic memorization Analysis with PrivacyGuard

## Introduction

We showcase a probabalistic memorization analysis using logits and logrprobs attack using PrivacyGuard.
Probabilistic memorization assessment measures the probability that a given LLM places on some target text content given a prompt, which
can be used as a proxy to quantify memorization of that text.

This tutorial will walk through the process of
- Using PrivacyGuard's generation tooling to conduct extraction evals on small LLMs
- Running LogprobsAttack and ProbabilisticMemorizationAnalysis to measure extraction rates of the ENRON email dataset.
- Running LogitsAttack and ProbabilisticMemorizationFromLogitsAnalysis to measure extraction rates of the ENRON email dataset.


In [1]:
%env CUDA_VISIBLE_DEVICES=1

env: CUDA_VISIBLE_DEVICES=1


In [2]:
import os

working_directory = "~/privacy_guard_working_directory"

working_directory_path = os.path.expanduser(working_directory)
if not os.path.isdir(working_directory_path):
    os.mkdir(working_directory_path)
else:
    print(f"Working directory already exists: {working_directory_path}")

Working directory already exists: /home/knchadha/privacy_guard_working_directory


# Preparing the Enron Email dataset

In each experiment, we
measure extraction rates with respect to 10,000 examples drawn from the Enron dataset, 
which is contained in the Pile (Gao et al., 2020)—the training
dataset for both Pythia and GPT-Neo 1.3B

To begin, download the May 7, 2015 version of the Enron dataset from https://www.cs.cmu.edu/~enron/

Move the compressed file to ~/privacy_guard_working_directory, and decompress with the following command. 
(NOTE that the dataset is large, so decompressing will create a large nexted directory)
```
cd ~/privacy_guard_working_directory
ls # Verify that enron_mail_20150507.tar.gz is located in the working directory
tar -xvzf enron_mail_20150507.tar.gz
```

In unix, then decompress the file with 'tar -xvzf enron_mail_20150507.tar.gz'

Once complete, check the directory structure
```
ls maildir
```



Next, we'll load samples from the decompressed dataset to use in extraction testing. 

maildir/allen-p/_sent_mail/ is a directory, containing ~600 emails

In [3]:
from typing import Dict, List

import pandas as pd

# Defining variables for setting up extraction samples
max_num_samples = 10
prompt_length_characters = 200
target_length_characters = 200
sample_length = prompt_length_characters + target_length_characters


# Pointing to samples to test extraction
example_content_dir = working_directory_path + "/maildir/allen-p/_sent_mail/"
extraction_targets: List[Dict[str, str]] = []


num_targets = 0
for filename in sorted(os.listdir(example_content_dir)):
    file_path = os.path.join(example_content_dir, filename)

    if os.path.isfile(file_path) and os.path.getsize(file_path) >= sample_length:
        with open(file_path, "r") as file:
            file_content = file.read()
            print(len(file_content[0:prompt_length_characters]))
            extraction_targets.append(
                {
                    "prompt": file_content[0:prompt_length_characters],
                    "target": file_content[
                        prompt_length_characters : prompt_length_characters
                        + target_length_characters
                    ],
                    "filename": filename,
                }
            )
        num_targets += 1
        if num_targets >= max_num_samples:
            break


print(f"Prepared extraction target with length: {len(extraction_targets)}")

extraction_targets_df = pd.DataFrame(extraction_targets)

200
200
200
200
200
200
200
200
200
200
Prepared extraction target with length: 10


In [4]:
# Save the dataframe to a .jsonl file
from privacy_guard.attacks.extraction.utils.data_utils import save_results

extraction_targets_path = working_directory_path + "/extraction_targets.jsonl"

if not os.path.isfile(extraction_targets_path):
    save_results(
        extraction_targets_df,
        extraction_targets_path,
        format="jsonl",
    )

    print(f"Saved extraction targets to jsonl file {extraction_targets_path}")
else:
    print(f"Extraction target file already exists as {extraction_targets_path}")

I0929 160229.414 _utils_internal.py:295] NCCL_DEBUG env var is set to None


I0929 160229.417 _utils_internal.py:304] NCCL_DEBUG is WARN from /etc/nccl.conf


Extraction target file already exists as /home/knchadha/privacy_guard_working_directory/extraction_targets.jsonl


# Define the Predictor

Extraction targets df is now prepared to run extraction attacks for memorization assessments, where we calculate the probability the model places on particular targts given the prompts. To start with, we define a Predictor object which loads the model and its corresponding tokenizer

This next step will use PrivacyGuard to load the Pythia model. 
(Note: this step will take some time)




In [5]:
from bento import fwdproxy
from privacy_guard.attacks.extraction.predictors.huggingface_predictor import (
    HuggingFacePredictor,
)

# 1) Create a HuggingFace predictor instance using the defined class
model_name = "EleutherAI/pythia-12b"

print(f"Loading model '{model_name}' using HuggingFacePredictor...")
with fwdproxy():
    huggingface_predictor = HuggingFacePredictor(
        model_name=model_name,
        device="cuda",
        model_kwargs={"torch_dtype": "auto"},  # Use appropriate dtype
        tokenizer_kwargs={},
    )

print(f"Loaded model '{huggingface_predictor.model_name}' from HuggingFace")

Loading model 'EleutherAI/pythia-12b' using HuggingFacePredictor...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loaded model 'EleutherAI/pythia-12b' from HuggingFace


[W 250929 16:02:55 tokenization_utils_base:1601] `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884


# Prepare and Execute LogprobsAttack

1. Prepare the LogprobsAttack
2. Execute the LogprobsAttack using "run_attack"

After executing this tutorial, feel free to clone and experiment with other models and datasets. 

In [6]:
from privacy_guard.attacks.extraction.logprobs_attack import LogprobsAttack

logprobs_attack = LogprobsAttack(
    input_file=extraction_targets_path,  # The dataset to perform logprobs attack on
    output_file=None,  # When specified, saves logprobs to file.
    predictor=huggingface_predictor,  # Pass the predictor instead of model/tokenizer
    prompt_column="prompt",  # Column used as prompt for each logprob extraction
    target_column="target",  # Column containing target text for logprob calculation
    output_column="prediction_logprobs",
    batch_size=4,
    temperature=1.1,
)

2025-09-29 16:02:55,652 - privacy_guard.attacks.extraction.logprobs_attack - WARNING - Logprobs attack argument "output_format" is unused when output file is not specified


2025-09-29 16:02:55,653 - privacy_guard.attacks.extraction.logprobs_attack - INFO - Loading data from /home/knchadha/privacy_guard_working_directory/extraction_targets.jsonl


2025-09-29 16:02:55,656 - privacy_guard.attacks.extraction.logprobs_attack - INFO - Loaded 10 rows


2025-09-29 16:02:55,657 - privacy_guard.attacks.extraction.logprobs_attack - INFO - Logprobs attack is ready to run


# Running LogprobsAttack

Now that LogprobsAttack has been configured and initialized, the we can perform the logproibs attack which calculates the log probabilities using "run_attack"

In [7]:
attack_result = logprobs_attack.run_attack()

2025-09-29 16:02:56,172 - privacy_guard.attacks.extraction.logprobs_attack - INFO - Executing logprobs attack


2025-09-29 16:02:56,178 - privacy_guard.attacks.extraction.logprobs_attack - INFO - Extracting logprobs for 10 prompt-target pairs


Computing logits:   0%|          | 0/3 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


I0929 160257.922 structured_logging.py:570] TritonTraceHandler: disabled because /logs does not exist


W0929 160257.924 triton_util.py:167] ===========[FB_TRITON]=========== Triton Version: 3.3.2+fb


Computing logits: 100%|██████████| 3/3 [00:05<00:00,  1.83s/it]


2025-09-29 16:03:01,764 - privacy_guard.attacks.extraction.logprobs_attack - INFO - Processing complete


2025-09-29 16:03:01,765 - privacy_guard.attacks.extraction.logprobs_attack - INFO - No output file specified, not saving results to disk


# Analysis

Now that the log probability calculation through logprobs_attack is complete, we can perform Privacy Analysis to compute do a memorization assessment of the dataset. 

In [8]:
from typing import Any, Dict, List

import pandas as pd

from IPython.display import display, Markdown

from privacy_guard.analysis.extraction.probabilistic_memorization_analysis_node import (
    ProbabilisticMemorizationAnalysisNode,
)

# Remove this line as it's not needed for LogprobsAttack result
# attack_result.lcs_bound_config = None

analysis_node = ProbabilisticMemorizationAnalysisNode(analysis_input=attack_result)

results = analysis_node.run_analysis()

# Update to use the new outputs from ProbabilisticMemorizationAnalysisNode
displays = []

def display_result(displays: List[Dict[str, Any]], augmented_row):
    displays.append(
        {
            "model_probability": augmented_row["model_probability"],
            "above_threshold": augmented_row["above_probability_threshold"],
            "n_probabilities": augmented_row.get("n_probabilities", "N/A"),
            "target": augmented_row["target"],
            "logprobs": augmented_row["prediction_logprobs"],
        }
    )

for augmented_row in results.augmented_output_dataset.T.to_dict().values():
    display_result(displays=displays, augmented_row=augmented_row)

display(pd.DataFrame(displays))

100%|██████████| 10/10 [00:00<00:00, 11302.36it/s]


100%|██████████| 10/10 [00:00<00:00, 34635.05it/s]


,model_probability,above_threshold,n_probabilities,target,logprobs
0,0.000000e+00,False,N/A,ext/plain; charset=us-ascii\nContent-Transfer-...,"[-20.359375, -19.515625, -20.34375, -16.734375..."
1,0.000000e+00,False,N/A,pe: text/plain; charset=us-ascii\nContent-Tran...,"[-25.515625, -12.0859375, -19.484375, -19.4687..."
2,0.000000e+00,False,N/A,nt-Type: text/plain; charset=us-ascii\nContent...,"[-22.03125, -18.59375, -15.390625, -17.078125,..."
3,0.000000e+00,False,N/A,text/plain; charset=us-ascii\nContent-Transfer...,"[-20.53125, -19.421875, -20.546875, -18.359375..."
4,0.000000e+00,False,N/A,t-Type: text/plain; charset=us-ascii\nContent-...,"[-17.875, -18.796875, -15.859375, -17.640625, ..."
5,0.000000e+00,False,N/A,t-Type: text/plain; charset=us-ascii\nContent-...,"[-18.25, -18.6875, -15.65625, -17.609375, -17...."
6,6.223975e-13,False,N/A,Version: 1.0\nContent-Type: text/plain; charse...,"[-0.01177978515625, -0.0016679763793945312, -2..."
7,1.688191e-20,False,N/A,ime-Version: 1.0\nContent-Type: text/plain; ch...,"[-0.669921875, -0.0012645721435546875, -0.0110..."
8,0.000000e+00,False,N/A,e-Version: 1.0\nContent-Type: text/plain; char...,"[-16.359375, -20.859375, -23.28125, -22.359375..."
9,2.888364e-21,False,N/A,gas price terms\nMime-Version: 1.0\nContent-T...,"[-6.72265625, -2.658203125, -5.8984375, -1.148..."


# Preparing and Executing LogitsAttack

1. Prepare the LogitsAttack
2. Execute the LogitsAttack using "run_attack"

In [9]:
from privacy_guard.attacks.extraction.logits_attack import LogitsAttack

# 2) Prepare the LogprobsAttack
logits_attack = LogitsAttack(
    input_file=extraction_targets_path,  # The dataset to perform logprobs attack on
    output_file=None,  # When specified, saves logprobs to file.
    predictor=huggingface_predictor,  # Pass the predictor instead of model/tokenizer
    prompt_column="prompt",  # Column used as prompt for each logprob extraction
    target_column="target",  # Column containing target text for logprob calculation
    output_column="prediction_logits",
    batch_size=4,
    temperature=1.1,
)

2025-09-29 16:03:02,746 - privacy_guard.attacks.extraction.logits_attack - WARNING - Logits attack argument "output_format" is unused when output file is not specified


2025-09-29 16:03:02,747 - privacy_guard.attacks.extraction.logits_attack - INFO - Loading data from /home/knchadha/privacy_guard_working_directory/extraction_targets.jsonl


2025-09-29 16:03:02,749 - privacy_guard.attacks.extraction.logits_attack - INFO - Loaded 10 rows


2025-09-29 16:03:02,750 - privacy_guard.attacks.extraction.logits_attack - INFO - Logits attack is ready to run


# Running LogitsAttack

Now that LogitsAttack has been configured and initialized, the we can perform the generation attack using "run_attack"

In [10]:
attack_result = logits_attack.run_attack()

2025-09-29 16:03:03,140 - privacy_guard.attacks.extraction.logits_attack - INFO - Executing logits attack


2025-09-29 16:03:03,141 - privacy_guard.attacks.extraction.logits_attack - INFO - Extracting logits for 10 prompt-target pairs


Computing logits: 100%|██████████| 3/3 [00:00<00:00, 11.34it/s]


2025-09-29 16:03:05,060 - privacy_guard.attacks.extraction.logits_attack - INFO - Tokenizing targets to create target_tokens column


2025-09-29 16:03:05,066 - privacy_guard.attacks.extraction.logits_attack - INFO - Processing complete


2025-09-29 16:03:05,067 - privacy_guard.attacks.extraction.logits_attack - INFO - No output file specified, not saving results to disk


# Analysis

Now that the generation attack is complete, we can perform Privacy Analysis to compute the extraction rate of the dataset. 

We'll look at the longest common substring score for each sample in the dataset, alonside the % of the target extracted. 

In [11]:
from typing import Any, Dict, List

import pandas as pd
from IPython.display import display, Markdown

from privacy_guard.analysis.extraction.probabilistic_memorization_analysis_from_logits_node import (
    ProbabilisticMemorizationAnalysisFromLogitsNode,
)

# Remove this line as it's not needed for LogprobsAttack result
# attack_result.lcs_bound_config = None

analysis_node_logits = ProbabilisticMemorizationAnalysisFromLogitsNode(analysis_input=attack_result)

results = analysis_node_logits.run_analysis()

print("Analysis run completed.")
# Update to use the new outputs from ProbabilisticMemorizationAnalysisNode
displays = []

def display_result(displays: List[Dict[str, Any]], augmented_row):
    displays.append(
        {
            "model_probability": augmented_row["model_probability"],
            "above_threshold": augmented_row["above_probability_threshold"],
            "n_probabilities": augmented_row.get("n_probabilities", "N/A"),
            "target": augmented_row["target"],
        }
    )

for augmented_row in results.augmented_output_dataset.T.to_dict().values():
    display_result(displays=displays, augmented_row=augmented_row)

display(pd.DataFrame(displays))

100%|██████████| 10/10 [00:03<00:00,  3.10it/s]


100%|██████████| 10/10 [00:00<00:00, 32263.88it/s]

Analysis run completed.


,model_probability,above_threshold,n_probabilities,target
0,0.000000e+00,False,N/A,ext/plain; charset=us-ascii\nContent-Transfer-...
1,0.000000e+00,False,N/A,pe: text/plain; charset=us-ascii\nContent-Tran...
2,0.000000e+00,False,N/A,nt-Type: text/plain; charset=us-ascii\nContent...
3,0.000000e+00,False,N/A,text/plain; charset=us-ascii\nContent-Transfer...
4,0.000000e+00,False,N/A,t-Type: text/plain; charset=us-ascii\nContent-...
5,0.000000e+00,False,N/A,t-Type: text/plain; charset=us-ascii\nContent-...
6,6.222453e-13,False,N/A,Version: 1.0\nContent-Type: text/plain; charse...
7,1.699094e-20,False,N/A,ime-Version: 1.0\nContent-Type: text/plain; ch...
8,0.000000e+00,False,N/A,e-Version: 1.0\nContent-Type: text/plain; char...
9,2.874247e-21,False,N/A,gas price terms\nMime-Version: 1.0\nContent-T...
